<a href="https://colab.research.google.com/github/VishnuGupta5883/appliedai/blob/master/Call_Backs_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<pre>
1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>

2. Code the model to classify data like below image

<img src='https://i.imgur.com/33ptOFy.png'>

3. Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.

4. Save your model at every epoch if your validation accuracy is improved from previous epoch. 

5. you have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
6. If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

7. You have to stop the training if your validation accuracy is not increased in last 2 epochs.

8. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

9. use cross entropy as loss function

10. Try the architecture params as given below. 
</pre>

<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
data=pd.read_csv('/content/drive/My Drive/Colab Notebooks/data.csv')
data.head()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


,f1,f2,label
0,0.450564,1.074305,0.0
1,0.085632,0.967682,0.0
2,0.117326,0.971521,1.0
3,0.982179,-0.380408,0.0
4,-0.720352,0.955850,0.0


#2.1 Callback function

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
import random as rn

X = data.drop(['label'], axis=1).values
Y = data['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, stratify=Y) 
#X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.33, stratify=y_train)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(13400, 2)
(13400,)
(6600, 2)
(6600,)


In [3]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
#from keras.callbacks import Callback

'''def getf1(X_test,y_test):
  val_f1 = []
  y_pred = model.predict(X_test)
  f1 = f1_score(y_test, y_pred,average='micro')
  val_f1.append(f1)
  print(val_f1)'''

class Metrics(tf.keras.callbacks.Callback):

   def on_train_begin(self, logs={}):
     self.val_f1s = []
     

   def on_epoch_end(self, epoch, logs={}):
     #val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
     val_predict = (np.asarray(self.model.predict(X_test))).round()
     #val_targ = self.model.validation_data[1]
     _val_f1 = f1_score(y_test, val_predict,average='micro') 
     self.val_f1s.append(_val_f1)
     #print(" value f1 ",_val_f1)
     print("  f1_score: "+"{:.4f}".format(_val_f1)); 
     
     return

history_own=Metrics()
#print(history_own.val_f1s)

In [5]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(5,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(input_layer)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer1)

#Creating a model
model = Model(inputs=input_layer,outputs=output)


#Callbacks
#history_own = LossHistory()
history_own  = Metrics()

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

model.compile(optimizer=optimizer, loss='BinaryCrossentropy',metrics=['AUC'])


model.fit(X_train,y_train,epochs=5, validation_data=(X_test,y_test), batch_size=20, callbacks=[history_own])


Epoch 1/5
670/670 [==============================] - 2s 3ms/step - loss: 0.7175 - auc: 0.5006 - val_loss: 0.6958 - val_auc: 0.4936
Epoch 2/5
670/670 [==============================] - 1s 2ms/step - loss: 0.6924 - auc: 0.5105 - val_loss: 0.6917 - val_auc: 0.5099
Epoch 3/5
670/670 [==============================] - 1s 2ms/step - loss: 0.6908 - auc: 0.5650 - val_loss: 0.6906 - val_auc: 0.5706
Epoch 4/5
670/670 [==============================] - 1s 2ms/step - loss: 0.6897 - auc: 0.5794 - val_loss: 0.6895 - val_auc: 0.5922
Epoch 5/5
670/670 [==============================] - 1s 2ms/step - loss: 0.6886 - auc: 0.5950 - val_loss: 0.6886 - val_auc: 0.5940


In [6]:
history_own.val_f1s

[0.5040909090909091,
 0.5324242424242425,
 0.5384848484848485,
 0.5868181818181818,
 0.5816666666666667]

## 2.2 Termiate training If getting any NaN values(either weigths or loss) while training

In [7]:
class TerminateNaN(tf.keras.callbacks.Callback):
        
    def on_epoch_end(self, epoch, logs={}):
        loss = logs.get('loss')
        if loss is not None:
            if np.isnan(loss) or np.isinf(loss):
                print("Invalid loss and terminated at epoch {}".format(epoch))
                self.model.stop_training = True

In [8]:
terminate= TerminateNaN()

model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test),batch_size=20,callbacks=[terminate])


Epoch 1/5
670/670 [==============================] - 1s 2ms/step - loss: 0.6875 - auc: 0.5998 - val_loss: 0.6876 - val_auc: 0.5933
Epoch 2/5
670/670 [==============================] - 1s 2ms/step - loss: 0.6864 - auc: 0.6012 - val_loss: 0.6866 - val_auc: 0.5941
Epoch 3/5
670/670 [==============================] - 1s 2ms/step - loss: 0.6854 - auc: 0.6031 - val_loss: 0.6855 - val_auc: 0.5978
Epoch 4/5
670/670 [==============================] - 1s 2ms/step - loss: 0.6842 - auc: 0.6039 - val_loss: 0.6845 - val_auc: 0.5987
Epoch 5/5
670/670 [==============================] - 1s 2ms/step - loss: 0.6832 - auc: 0.6046 - val_loss: 0.6835 - val_auc: 0.5982


## 2.3 ModelCheckpoint - Save the model after every epoch.

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint

#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(5,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(input_layer)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer1)
#Creating a model
model = Model(inputs=input_layer,outputs=output)


#Callbacks
#file path, it saves the model in the 'model_save' folder and we are naming model with epoch number 
#and val auc to differtiate with other models
#you have to create model_save folder before running the code.
filepath="model_save/weights-{epoch:02d}-{val_auc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_auc',  verbose=1, save_best_only=True, mode='max')

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

model.compile(optimizer=optimizer, loss='BinaryCrossentropy',metrics=['AUC'])


model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test),batch_size=20,callbacks=[checkpoint])

Epoch 1/5
641/670 [===========================>..] - ETA: 0s - loss: 0.6989 - auc: 0.4993
Epoch 00001: val_auc improved from -inf to 0.49541, saving model to model_save/weights-01-0.50.hdf5
670/670 [==============================] - 1s 2ms/step - loss: 0.6987 - auc: 0.4997 - val_loss: 0.6944 - val_auc: 0.4954
Epoch 2/5
654/670 [============================>.] - ETA: 0s - loss: 0.6934 - auc: 0.4974
Epoch 00002: val_auc improved from 0.49541 to 0.50885, saving model to model_save/weights-02-0.51.hdf5
670/670 [==============================] - 1s 2ms/step - loss: 0.6934 - auc: 0.4981 - val_loss: 0.6931 - val_auc: 0.5088
Epoch 3/5
649/670 [============================>.] - ETA: 0s - loss: 0.6931 - auc: 0.5050
Epoch 00003: val_auc did not improve from 0.50885
670/670 [==============================] - 1s 2ms/step - loss: 0.6931 - auc: 0.5044 - val_loss: 0.6929 - val_auc: 0.5035
Epoch 4/5
645/670 [===========================>..] - ETA: 0s - loss: 0.6929 - auc: 0.5077
Epoch 00004: val_auc imp

## 2.4 You have to stop the training if your validation accuracy is not increased in last 2 epochs.

In [17]:
from tensorflow.keras.callbacks import EarlyStopping

#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(5,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(input_layer)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer1)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

earlystop = EarlyStopping(monitor='val_auc', patience=2, verbose=1, mode='max')


optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

model.compile(optimizer=optimizer, loss='BinaryCrossentropy',metrics=['AUC'])


model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test),batch_size=20,callbacks=[earlystop])


Epoch 1/5
670/670 [==============================] - 1s 2ms/step - loss: 0.7010 - auc: 0.5011 - val_loss: 0.6948 - val_auc: 0.4931
Epoch 2/5
670/670 [==============================] - 2s 3ms/step - loss: 0.6935 - auc: 0.5031 - val_loss: 0.6932 - val_auc: 0.5000
Epoch 3/5
670/670 [==============================] - 1s 2ms/step - loss: 0.6930 - auc: 0.5048 - val_loss: 0.6929 - val_auc: 0.5140
Epoch 4/5
670/670 [==============================] - 1s 2ms/step - loss: 0.6929 - auc: 0.5202 - val_loss: 0.6929 - val_auc: 0.5129
Epoch 5/5
670/670 [==============================] - 1s 2ms/step - loss: 0.6928 - auc: 0.5165 - val_loss: 0.6927 - val_auc: 0.5390


## 2.5 Learning Rate Schedular

 you have to decay learning based on below conditions
 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.

In [21]:
from tensorflow.keras.callbacks import LearningRateScheduler

def changeLearningRate(epoch):
    initial_learningrate=0.01
    if epoch % 3 ==0:
      changed = initial_learningrate*(1-0.05)**epoch
    else:
      changed = initial_learningrate*(1-0.1)**epoch
    return changed

changed_lr = []
for i in range(1,10):
  changed_lr.append(changeLearningRate(i))
  


#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(5,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(input_layer)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer1)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

lrschedule = LearningRateScheduler(changeLearningRate, verbose=0.1)
filepath="model_save/weights-{epoch:02d}-{val_auc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_auc',  verbose=1, save_best_only=True, mode='max')
earlystop = EarlyStopping(monitor='val_auc', patience=2, verbose=1, mode='max')

# here we are creating a list with all the callbacks we want
callback_list = [history_own,lrschedule, earlystop, checkpoint,terminate]

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

model.compile(optimizer=optimizer, loss='BinaryCrossentropy',metrics=['AUC'])


model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),batch_size=20,callbacks=[callback_list])



Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
Epoch 1/20
669/670 [============================>.] - ETA: 0s - loss: 0.7250 - auc: 0.5025  f1_score: 0.4965

Epoch 00001: val_auc improved from -inf to 0.49247, saving model to model_save/weights-01-0.4925.hdf5
670/670 [==============================] - 2s 2ms/step - loss: 0.7249 - auc: 0.5025 - val_loss: 0.6964 - val_auc: 0.4925 - lr: 0.0100

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009000000000000001.
Epoch 2/20
648/670 [============================>.] - ETA: 0s - loss: 0.6930 - auc: 0.5093  f1_score: 0.5159

Epoch 00002: val_auc improved from 0.49247 to 0.52159, saving model to model_save/weights-02-0.5216.hdf5
670/670 [==============================] - 1s 2ms/step - loss: 0.6930 - auc: 0.5092 - val_loss: 0.6922 - val_auc: 0.5216 - lr: 0.0090

Epoch 00003: LearningRateScheduler reducing learning rate to 0.008100000000000001.
Epoch 3/20
641/670 [===========================>..] - ETA: 0s - loss:

## Observation

## 1. on epoch-4 f1_score is maximum-0.5892 --> val_auc= 0.5939
## 2. Val_loss is decreasing on every epoch. Initially loss is decreasing significant but after few epoch loss change is not significant.
##3. After epoch 10 val_auc is not increasing.

# Model-2

1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 

In [22]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(5,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(input_layer)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer1)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

lrschedule = LearningRateScheduler(changeLearningRate, verbose=0.1)
filepath="model_save/weights-{epoch:02d}-{val_auc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_auc',  verbose=1, save_best_only=True, mode='max')
earlystop = EarlyStopping(monitor='val_auc', patience=2, verbose=1, mode='max')

# here we are creating a list with all the callbacks we want
callback_list = [history_own,lrschedule, earlystop, checkpoint,terminate]

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

model.compile(optimizer=optimizer, loss='BinaryCrossentropy',metrics=['AUC'])


model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),batch_size=20,callbacks=[callback_list])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
Epoch 1/20
656/670 [============================>.] - ETA: 0s - loss: 0.7357 - auc: 0.4643  f1_score: 0.4562

Epoch 00001: val_auc improved from -inf to 0.45563, saving model to model_save/weights-01-0.4556.hdf5
670/670 [==============================] - 2s 2ms/step - loss: 0.7352 - auc: 0.4640 - val_loss: 0.7080 - val_auc: 0.4556 - lr: 0.0100

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009000000000000001.
Epoch 2/20
650/670 [============================>.] - ETA: 0s - loss: 0.7017 - auc: 0.4626  f1_score: 0.4409

Epoch 00002: val_auc did not improve from 0.45563
670/670 [==============================] - 1s 2ms/step - loss: 0.7016 - auc: 0.4620 - val_loss: 0.6979 - val_auc: 0.4517 - lr: 0.0090

Epoch 00003: LearningRateScheduler reducing learning rate to 0.008100000000000001.
Epoch 3/20
647/670 [===========================>..] - ETA: 0s - loss: 0.6955 - auc: 0.4580  f1_score: 0.4935

Epoch 00003: v

# Observations

## 1. f1_score is increasing on every epoch - max -f1_score= 0.5164 , val_auc= 0.5321
## 2. Val_loss change is very less (not significant) on every epoch. 
## 3. After epoch 7 val_auc is not increasing.

# Model-3
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 

In [23]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(5,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(input_layer)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.he_uniform())(layer1)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

lrschedule = LearningRateScheduler(changeLearningRate, verbose=0.1)
filepath="model_save/weights-{epoch:02d}-{val_auc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_auc',  verbose=1, save_best_only=True, mode='max')
earlystop = EarlyStopping(monitor='val_auc', patience=2, verbose=1, mode='max')

# here we are creating a list with all the callbacks we want
callback_list = [history_own,lrschedule, earlystop, checkpoint,terminate]

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

model.compile(optimizer=optimizer, loss='BinaryCrossentropy',metrics=['AUC'])


model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),batch_size=20,callbacks=[callback_list])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
Epoch 1/20
656/670 [============================>.] - ETA: 0s - loss: 0.7529 - auc: 0.5103  f1_score: 0.5164

Epoch 00001: val_auc improved from -inf to 0.51928, saving model to model_save/weights-01-0.5193.hdf5
670/670 [==============================] - 2s 2ms/step - loss: 0.7524 - auc: 0.5094 - val_loss: 0.7034 - val_auc: 0.5193 - lr: 0.0100

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009000000000000001.
Epoch 2/20
638/670 [===========================>..] - ETA: 0s - loss: 0.6968 - auc: 0.5218  f1_score: 0.5185

Epoch 00002: val_auc improved from 0.51928 to 0.53082, saving model to model_save/weights-02-0.5308.hdf5
670/670 [==============================] - 1s 2ms/step - loss: 0.6964 - auc: 0.5225 - val_loss: 0.6899 - val_auc: 0.5308 - lr: 0.0090

Epoch 00003: LearningRateScheduler reducing learning rate to 0.008100000000000001.
Epoch 3/20
660/670 [============================>.] - ETA: 0s - loss:

# Observation

## 1. f1_score and val_auc is increasing on every epoch. max -f1_score = 0.5639  val_auc= 0.6141
## 2. Val_loss is decreasing on every epoch. Initially loss is decreasing significant but after few epoch loss change is not significant.


# Model-4

1. Try with any values to get better accuracy/f1 score. 

In [27]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(5,activation='selu',kernel_initializer=tf.keras.initializers.he_uniform())(input_layer)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.he_uniform())(layer1)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

lrschedule = LearningRateScheduler(changeLearningRate, verbose=0.1)
filepath="model_save/weights-{epoch:02d}-{val_auc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_auc',  verbose=1, save_best_only=True, mode='max')
earlystop = EarlyStopping(monitor='val_auc', patience=2, verbose=1, mode='max')

# here we are creating a list with all the callbacks we want
callback_list = [history_own,lrschedule, earlystop, checkpoint,terminate]

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

model.compile(optimizer=optimizer, loss='BinaryCrossentropy',metrics=['AUC'])


model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test),batch_size=20,callbacks=[callback_list])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
Epoch 1/20
652/670 [============================>.] - ETA: 0s - loss: 0.6896 - auc: 0.5760  f1_score: 0.6114

Epoch 00001: val_auc improved from -inf to 0.66031, saving model to model_save/weights-01-0.6603.hdf5
670/670 [==============================] - 2s 2ms/step - loss: 0.6893 - auc: 0.5773 - val_loss: 0.6751 - val_auc: 0.6603 - lr: 0.0100

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009000000000000001.
Epoch 2/20
653/670 [============================>.] - ETA: 0s - loss: 0.6730 - auc: 0.6596  f1_score: 0.6058

Epoch 00002: val_auc did not improve from 0.66031
670/670 [==============================] - 1s 2ms/step - loss: 0.6729 - auc: 0.6598 - val_loss: 0.6719 - val_auc: 0.6591 - lr: 0.0090

Epoch 00003: LearningRateScheduler reducing learning rate to 0.008100000000000001.
Epoch 3/20
664/670 [============================>.] - ETA: 0s - loss: 0.6694 - auc: 0.6595  f1_score: 0.6089

Epoch 00003: v

# Observation

## 1. Using 'selu' as activation layer val_auc is increasing at every epoch. max -val_acu =.7032 and f1_score = 0.6397

##2 val_loss is decreasing on every epoch.